In [1]:
import pandas as pd
import os
import numpy as np
import re
import sys
from pandas import DataFrame
from sklearn.utils import shuffle
from collections import OrderedDict
import hashlib


In [2]:
from Extractor import FeatureExtractor

In [9]:
APACHE_ACCESS_LOG_PATTERN = '^(\S+) (\S+) (\S+) (\S+) (\S+) (.*)'


In [10]:
fo = open("abc.txt", "r")    

In [11]:
fo = fo.read().decode('utf-8')

In [12]:
fo 

u'081109 203518 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.19.102:54106 dest: /10.250.19.102:50010\r\n081109 203518 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.allocateBlock: /mnt/hadoop/mapred/system/job_200811092030_0001/job.jar. blk_-1608999687919862906\r\n081109 203519 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.10.6:40524 dest: /10.250.10.6:50010\r\n081109 203519 145 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.14.224:42420 dest: /10.250.14.224:50010\r\n081109 203519 145 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_-1608999687919862906 terminating\r\n081109 203519 145 INFO dfs.DataNode$PacketResponder: PacketResponder 2 for block blk_-1608999687919862906 terminating\r\n\r\n\r\n'

In [13]:
def parse_hdfs_file(file_ab, APACHE_ACCESS_LOG_PATTERN):
    ab = []
    file_line = file_ab.split("\n")
    for line in range(len(file_line)):
        match = re.findall(APACHE_ACCESS_LOG_PATTERN,file_line[line])
        for mat in range(len(match)):
            bd = re.sub("[\d.-]+", "<*>", match[mat][5])
            eventid = hashlib.md5(' '.join(bd).encode('utf-8')).hexdigest()[0:8]
            ab.append([match[mat][0],match[mat][1],match[mat][2],match[mat][3],match[mat][4],match[mat][5],bd, eventid] )
    return ab 
    

In [14]:
parse_hdfs_file(fo, APACHE_ACCESS_LOG_PATTERN)

[[u'081109',
  u'203518',
  u'143',
  u'INFO',
  u'dfs.DataNode$DataXceiver:',
  u'Receiving block blk_-1608999687919862906 src: /10.250.19.102:54106 dest: /10.250.19.102:50010\r',
  u'Receiving block blk_<*> src: /<*>:<*> dest: /<*>:<*>\r',
  '91205f34'],
 [u'081109',
  u'203518',
  u'35',
  u'INFO',
  u'dfs.FSNamesystem:',
  u'BLOCK* NameSystem.allocateBlock: /mnt/hadoop/mapred/system/job_200811092030_0001/job.jar. blk_-1608999687919862906\r',
  u'BLOCK* NameSystem<*>allocateBlock: /mnt/hadoop/mapred/system/job_<*>_<*>/job<*>jar<*> blk_<*>\r',
  'c164f67d'],
 [u'081109',
  u'203519',
  u'143',
  u'INFO',
  u'dfs.DataNode$DataXceiver:',
  u'Receiving block blk_-1608999687919862906 src: /10.250.10.6:40524 dest: /10.250.10.6:50010\r',
  u'Receiving block blk_<*> src: /<*>:<*> dest: /<*>:<*>\r',
  '91205f34'],
 [u'081109',
  u'203519',
  u'145',
  u'INFO',
  u'dfs.DataNode$DataXceiver:',
  u'Receiving block blk_-1608999687919862906 src: /10.250.14.224:42420 dest: /10.250.14.224:50010\r',

In [ ]:
# parse_hdfs_file(fo,APACHE_ACCESS_LOG_PATTERN)

In [ ]:
# fo = open("HDFS.log", "r")    

In [ ]:
# dta = parse_hdfs_file(fo)

In [ ]:
# data = pd.DataFrame(dta,columns=["Date","Time","Pid","Info","Component","Content","EventTemplate","EventId"])

In [ ]:
# data.head(10)

In [ ]:
def read_file(data):
    data_dict = OrderedDict()
    for idx, row in data.iterrows():
        blkId_list = re.findall(r'(blk_-?\d+)', row['Content'])
        blkId_set = set(blkId_list)
        for blk_Id in blkId_set:
            if not blk_Id in data_dict:
                data_dict[blk_Id] = []
            data_dict[blk_Id].append(row['EventId'])
    data_df = pd.DataFrame(list(data_dict.items()), columns=['BlockId', 'EventSequence'])
    
    return data_df

In [ ]:
def dict_label(df_train_append, label_file):    
    label_data =label_file.set_index('BlockId')
    label_dict = label_data['Label'].to_dict() 
    df_train_append['Label'] = df_train_append['BlockId'].apply(lambda x: 1 if label_dict[x] == 'Anomaly' else 0)
    return pd.DataFrame(df_train_append)



In [ ]:
# fo = open("abc.txt", "r")    

In [ ]:
# def read_file(fo , charset='utf-8'):
#     with open(fo , 'r') as f:
#         return f.read().decode(charset)

In [ ]:
# df_train_append = read_file(data)

In [ ]:
# fo

In [ ]:
# df_train_append.head(10)

In [ ]:
# label_csv = pd.read_csv('anomaly_label.csv')

In [ ]:
# label_csv.head(10)

In [ ]:
# label_data =label_csv.set_index('BlockId')
# label_dict = label_data['Label'].to_dict()
# label_dict

In [ ]:
# df_train_append

In [ ]:
# df_train_append['Label'] = df_train_append['BlockId'].apply(lambda x: 1 if label_dict[x] == 'Anomaly' else 0)

In [ ]:
# df_event = pd.DataFrame(df_train_append)

In [ ]:
# df_event.head(10)

In [ ]:
# from sklearn.model_selection import train_test_split

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(df_train_append['EventSequence'].values, df_train_append['Label'].values, test_size = 0.2) 

In [ ]:
# from collections import Counter
# from scipy.special import expit
# from itertools import compress

In [ ]:
# feature_extractor = FeatureExtractor()
# x_train = feature_extractor.fit_transform(x_train, term_weighting='tf-idf')
# x_test = feature_extractor.transform(x_test)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# rfclf = RandomForestClassifier(max_depth=1, random_state=42)
# rfclf = rfclf.fit(x_train, y_train)

# from sklearn.metrics import f1_score

# predicted_y = rfclf.predict(x_test)
# print("test accuracy: ",f1_score(y_test, predicted_y, average='micro'))

In [ ]:
# predicted_y

In [ ]:
# from sklearn.externals import joblib  

# joblib.dump(rfclf, "trained-model.pkl")